In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
import joblib

# Step 1: Read the CSV file
df = pd.read_csv('output.csv')

df['date'] = pd.to_datetime(df['date'], dayfirst=True).map(pd.Timestamp.toordinal)
# Normalize the total carbon emission amounts
scaler = MinMaxScaler(feature_range=(0, 1))
df['total_carbon_emission'] = scaler.fit_transform(df[['total_carbon_emission']])

# Step 3: Split the data
X = df['date'].values.reshape(-1, 1)
y = df['total_carbon_emission'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Create the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(1,)),
    Dense(64, activation='relu'),
    Dense(1)
])

# Step 5: Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Step 6: Train the model
model.fit(X_train, y_train, epochs=110, batch_size=32, validation_split=0.2)

# Step 7: Evaluate the model
model.evaluate(X_test, y_test)

joblib.dump(model, 'carbon_emission_model.pkl')

Epoch 1/110
5/5 [==============================] - 1s 53ms/step - loss: 59126772.0000 - val_loss: 2903096.5000
Epoch 2/110
5/5 [==============================] - 0s 11ms/step - loss: 21928908.0000 - val_loss: 894702.7500
Epoch 3/110
5/5 [==============================] - 0s 14ms/step - loss: 12593689.0000 - val_loss: 181538.8906
Epoch 4/110
5/5 [==============================] - 0s 14ms/step - loss: 6706731.5000 - val_loss: 1648853.5000
Epoch 5/110
5/5 [==============================] - 0s 15ms/step - loss: 3635914.5000 - val_loss: 2537653.7500
Epoch 6/110
5/5 [==============================] - 0s 10ms/step - loss: 2148946.7500 - val_loss: 2361325.7500
Epoch 7/110
5/5 [==============================] - 0s 19ms/step - loss: 1358832.5000 - val_loss: 1743128.0000
Epoch 8/110
5/5 [==============================] - 0s 14ms/step - loss: 870784.1250 - val_loss: 1135135.7500
Epoch 9/110
5/5 [==============================] - 0s 11ms/step - loss: 549710.7500 - val_loss: 679875.0625
Epoch 10/110

['carbon_emission_model.pkl']

In [5]:
import pandas as pd
import numpy as np
import joblib
from datetime import datetime, timedelta

# Load the model and the pre-fitted scaler
model = joblib.load('carbon_emission_model.pkl')
#scaler = joblib.load('scaler.pkl')  # Load the pre-fitted scaler

# Get user input
month_year_str = input("Enter month and year (YYYY-MM): ")
year, month = map(int, month_year_str.split('-'))

# Generate all Wednesdays of the specified month
first_day = datetime(year, month, 1)
wednesdays = [first_day + timedelta(days=(2 - first_day.weekday() + 7 * i)) for i in range(5) if (first_day + timedelta(days=(2 - first_day.weekday() + 7 * i))).month == month]

# Prepare the data for prediction
wednesday_ordinals = np.array([datetime.toordinal(wed) for wed in wednesdays]).reshape(-1, 1)
predicted_emissions = model.predict(wednesday_ordinals)
predicted_emissions = scaler.inverse_transform(predicted_emissions)

# Create a DataFrame to store the results
results = pd.DataFrame({
    'Date': [wed.strftime("%Y-%m-%d") for wed in wednesdays],
    'Predicted Carbon Emission': predicted_emissions.flatten()
})

# Save the results to a CSV file
csv_filename = f"predicted_emissions_{year}_{month:02d}.csv"
results.to_csv(csv_filename, index=False)

print(f"Predicted carbon emissions saved to {csv_filename}")

Enter month and year (YYYY-MM): 2027-06
1/1 [==============================] - 0s 55ms/step
Predicted carbon emissions saved to predicted_emissions_2027_06.csv


In [6]:
df1 = pd.read_csv('predicted_emissions_2027_06.csv')
df1.head()


,Date,Predicted Carbon Emission
0,2027-06-02,4267.9185
1,2027-06-09,4244.7485
2,2027-06-16,4269.6562
3,2027-06-23,4269.0770
4,2027-06-30,4266.7603
